In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00


In [2]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time
from torchinfo import summary

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-32a081f8d74d>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-32a081f8d74d>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [11]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("Best trial:")
# print(study.best_trial)

In [12]:
# best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8 * 2,
    learning_rate=6.369414513935845e-05,
    weight_decay= 0.00195428799,
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training time: {training_time:.2f} seconds")
print("<<==============================================>>")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 1410


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.432309,0.820690,0.360107,0.396552,0.372721
1,No log,0.386399,0.847215,0.654939,0.487685,0.536168
2,No log,0.323472,0.878515,0.778277,0.596059,0.656650
3,No log,0.324363,0.882228,0.805484,0.642857,0.691397
4,No log,0.349631,0.869496,0.749761,0.667488,0.692519
5,0.304200,0.318074,0.888594,0.759841,0.729064,0.737327
6,0.304200,0.326899,0.889655,0.766150,0.719212,0.740244
7,0.304200,0.357148,0.890716,0.758440,0.741379,0.747545
8,0.304200,0.390282,0.888594,0.762194,0.724138,0.737671
9,0.304200,0.381921,0.883820,0.748308,0.729064,0.735432


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
Saving model che

<<=================Training Time================>>
Training time: 1112.79 seconds
<<==============================================>>


In [20]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 377
  Batch size = 16


{'eval_loss': 0.4020530581474304, 'eval_accuracy': 0.8917771883289125, 'eval_precision': 0.7460858142113475, 'eval_recall': 0.7660098522167488, 'eval_f1': 0.7551173475083244, 'eval_runtime': 5.3438, 'eval_samples_per_second': 70.549, 'eval_steps_per_second': 4.491, 'epoch': 14.99}


In [21]:
!mkdir 'best_model_roberta-large_python'
!mkdir 'best_model_roberta-large_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_roberta-large_python')
tokenizer.save_pretrained('./best_model_roberta-large_python_tokenizer')

# Load model and tokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained('./best_model_roberta-large_python')
tokenizer = RobertaTokenizer.from_pretrained('./best_model_roberta-large_python_tokenizer')

mkdir: cannot create directory ‘best_model_roberta-large_python’: File exists
mkdir: cannot create directory ‘best_model_roberta-large_python_tokenizer’: File exists


Saving model checkpoint to ./best_model_roberta-large_python
Configuration saved in ./best_model_roberta-large_python/config.json
Model weights saved in ./best_model_roberta-large_python/pytorch_model.bin
tokenizer config file saved in ./best_model_roberta-large_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_roberta-large_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_roberta-large_python/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,


## Model Testing

In [22]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [23]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [24]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("<<==============================================>>")
print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 406
  Batch size = 16


<<==============================================>>
<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.8379310344827586, 'precision': 0.6356695106874367, 'recall': 0.6353211009174312, 'f1': 0.6334020835066998}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 5.88 seconds
<<==============================================>>


In [25]:
!zip -r best_model_roberta-large_python.zip './best_model_roberta-large_python'
!zip -r best_model_roberta-large_python_tokenizer.zip './best_model_roberta-large_python_tokenizer'

updating: best_model_roberta-large_python/ (stored 0%)
updating: best_model_roberta-large_python/pytorch_model.bin (deflated 15%)
updating: best_model_roberta-large_python/training_args.bin (deflated 51%)
updating: best_model_roberta-large_python/config.json (deflated 54%)
updating: best_model_roberta-large_python_tokenizer/ (stored 0%)
updating: best_model_roberta-large_python_tokenizer/vocab.json (deflated 63%)
updating: best_model_roberta-large_python_tokenizer/tokenizer_config.json (deflated 79%)
updating: best_model_roberta-large_python_tokenizer/merges.txt (deflated 53%)
updating: best_model_roberta-large_python_tokenizer/special_tokens_map.json (deflated 84%)


In [26]:
# Transfer the model to Google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_roberta-large_python.zip "/content/drive/MyDrive/nlbse/models/roberta-large_Python/"
!cp best_model_roberta-large_python_tokenizer.zip "/content/drive/MyDrive/nlbse/models/roberta-large_Python_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model_folder = '/content/drive/MyDrive/nlbse/models/roberta-large_Python/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/roberta-large_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/roberta-large_Python/best_model_roberta-large_python.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/roberta-large_Python_Tokenizer/best_model_roberta-large_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/nlbse/models/roberta-large_Python/best_model_roberta-large_python.zip
replace ./best_model_roberta-large_python/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_python/pytorch_model.bin  
  inflating: ./best_model_roberta-large_python/training_args.bin  
  inflating: ./best_model_roberta-large_python/config.json  
Archive:  /content/drive/MyDrive/nlbse/models/roberta-large_Python_Tokenizer/best_model_roberta-large_python_tokenizer.zip
replace ./best_model_roberta-large_python_tokenizer/vocab.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_python_tokenizer/vocab.json  
  inflating: ./best_model_roberta-large_python_tokenizer/tokenizer_config.json  
  inflating: ./best_model_roberta-large_python_tokenizer/merges.txt  
  inflating: ./best_model_roberta-large_python_tokenizer/special_tokens_map.json  


In [29]:
best_model_roberta_large_python = 'best_model_roberta-large_python'
best_model_roberta_large_python_tokenizer = 'best_model_roberta-large_python_tokenizer'

In [30]:
# Load the model and tokenizer from the checkpoint
model = RobertaForSequenceClassification.from_pretrained(best_model_roberta_large_python)
tokenizer = RobertaTokenizer.from_pretrained(best_model_roberta_large_python_tokenizer)

text = "for detecting automatically generated fields."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_roberta-large_python/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

SequenceClassifierOutput(loss=None, logits=tensor([[-3.0394, -4.7529,  3.4335, -4.3031, -4.7776]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3
